In [5]:
net_struct = {'alexnet': {'net':[[11,4,0],[3,2,0],[5,1,2],[3,2,0],[3,1,1],[3,1,1],[3,1,1],[3,2,0]],  
                   'name':['conv1','pool1','conv2','pool2','conv3','conv4','conv5','pool5']},  
       'vgg16': {'net':[[3,1,1],[3,1,1],[2,2,0],[3,1,1],[3,1,1],[2,2,0],[3,1,1],[3,1,1],[3,1,1],  
                        [2,2,0],[3,1,1],[3,1,1],[3,1,1],[2,2,0],[3,1,1],[3,1,1],[3,1,1],[2,2,0]],  
                 'name':['conv1_1','conv1_2','pool1','conv2_1','conv2_2','pool2','conv3_1','conv3_2',  
                         'conv3_3', 'pool3','conv4_1','conv4_2','conv4_3','pool4','conv5_1','conv5_2','conv5_3','pool5']},  
       'zf-5':{'net': [[7,2,3],[3,2,1],[5,2,2],[3,2,1],[3,1,1],[3,1,1],[3,1,1]],  
               'name': ['conv1','pool1','conv2','pool2','conv3','conv4','conv5']}}  
imsize = 227  

In [6]:
def outFromIn(isz, net, layernum):  
    totstride = 1  
    insize = isz  
    for layer in range(layernum):  
        fsize, stride, pad = net[layer]  
        outsize = (insize - fsize + 2*pad) / stride + 1  
        insize = outsize  
        totstride = totstride * stride  
    return outsize, totstride  
  
def inFromOut(net, layernum):  
    RF = 1  
    for layer in reversed(range(layernum)):  
        fsize, stride, pad = net[layer]  
        RF = ((RF -1)* stride) + fsize  
    return RF  

def topToBottom(net, layernum, top_layer):
    RF = 1  
    for layer in  reversed(range(layernum, top_layer)):  #当前层至底层
#         print 'in func {}'.format(layer)
        fsize, stride, pad = net[layer]  
        RF = ((RF -1)* stride) + fsize  
    return RF     

def bottomToTop(net, layernum):
    if layernum==1:  #从1开始计数的
        return net[0][0]
    r_in=net[0][0]
    j_in=net[0][1]
    for layer in (range(1,layernum)):  
        fsize, stride, pad = net[layer]  
        j_out =j_in*stride
        r_out = r_in + (fsize -1)*j_in 
        r_in=r_out
        j_in=j_out
    return r_out 

In [7]:
for net in net_struct.keys():  
        print ('************net structrue name is %s**************'% net)  
        for top_layer in reversed(range(len(net_struct[net]['net']))):
            for layernum in reversed(range(top_layer + 1)):
                rf_uToD = topToBottom(net_struct[net]['net'], layernum, top_layer+1)  
                print ("layernum %3d, Layer Name = %s, RF size: topToDown = %3d" % (layernum, net_struct[net]['name'][layernum], 
                                                                                                            rf_uToD))  

************net structrue name is vgg16**************
layernum  17, Layer Name = pool5, RF size: topToDown =   2
layernum  16, Layer Name = conv5_3, RF size: topToDown =   4
layernum  15, Layer Name = conv5_2, RF size: topToDown =   6
layernum  14, Layer Name = conv5_1, RF size: topToDown =   8
layernum  13, Layer Name = pool4, RF size: topToDown =  16
layernum  12, Layer Name = conv4_3, RF size: topToDown =  18
layernum  11, Layer Name = conv4_2, RF size: topToDown =  20
layernum  10, Layer Name = conv4_1, RF size: topToDown =  22
layernum   9, Layer Name = pool3, RF size: topToDown =  44
layernum   8, Layer Name = conv3_3, RF size: topToDown =  46
layernum   7, Layer Name = conv3_2, RF size: topToDown =  48
layernum   6, Layer Name = conv3_1, RF size: topToDown =  50
layernum   5, Layer Name = pool2, RF size: topToDown = 100
layernum   4, Layer Name = conv2_2, RF size: topToDown = 102
layernum   3, Layer Name = conv2_1, RF size: topToDown = 104
layernum   2, Layer Name = pool1, RF si

In [8]:
if __name__ == '__main__':  
    print ("layer output sizes given image = %dx%d" % (imsize, imsize))  
  
for net in net_struct.keys():  
        print ('************net structrue name is %s**************'% net)  
        for i in range(len(net_struct[net]['net'])):  
            p = outFromIn(imsize,net_struct[net]['net'], i+1)  
            rf_uToD = inFromOut(net_struct[net]['net'], i+1)  
            rf_dToU = bottomToTop(net_struct[net]['net'], i+1) 
            print ("Layer Name = %s, Output size = %3d, Stride = % 3d, RF size: topToDown = %3d, downToTop = %3d" % (net_struct[net]['name'][i], 
                                                                                                    p[0], p[1], rf_uToD, rf_dToU))  

layer output sizes given image = 227x227
************net structrue name is vgg16**************
Layer Name = conv1_1, Output size = 227, Stride =   1, RF size: topToDown =   3, downToTop =   3
Layer Name = conv1_2, Output size = 227, Stride =   1, RF size: topToDown =   5, downToTop =   5
Layer Name = pool1, Output size = 113, Stride =   2, RF size: topToDown =   6, downToTop =   6
Layer Name = conv2_1, Output size = 113, Stride =   2, RF size: topToDown =  10, downToTop =  10
Layer Name = conv2_2, Output size = 113, Stride =   2, RF size: topToDown =  14, downToTop =  14
Layer Name = pool2, Output size =  56, Stride =   4, RF size: topToDown =  16, downToTop =  16
Layer Name = conv3_1, Output size =  56, Stride =   4, RF size: topToDown =  24, downToTop =  24
Layer Name = conv3_2, Output size =  56, Stride =   4, RF size: topToDown =  32, downToTop =  32
Layer Name = conv3_3, Output size =  56, Stride =   4, RF size: topToDown =  40, downToTop =  40
Layer Name = pool3, Output size =  2